In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Tuple

class SparseAutoencoder(nn.Module):
    def __init__(self, input_dim: int, expansion_factor: float = 16):
        super().__init__()
        self.input_dim = input_dim
        self.latent_dim = int(input_dim * expansion_factor)
        self.decoder = nn.Linear(self.latent_dim, input_dim, bias=True)
        self.encoder = nn.Linear(input_dim, self.latent_dim, bias=True)

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        encoded = F.relu(self.encoder(x))
        decoded = self.decoder(encoded)
        return decoded, encoded

    def encode(self, x: torch.Tensor) -> torch.Tensor:
        with torch.no_grad():
            return F.relu(self.encoder(x))

    def decode(self, x: torch.Tensor) -> torch.Tensor:
        with torch.no_grad():
            return self.decoder(x)

    @classmethod
    def from_pretrained(cls, path: str, input_dim: int, expansion_factor: float = 16, device: str = "mps") -> "SparseAutoencoder":
        model = cls(input_dim=input_dim, expansion_factor=expansion_factor)
        state_dict = torch.load(path, map_location=device)
        model.load_state_dict(state_dict)
        model = model.to(device)
        model.eval()
        return model

In [3]:
from huggingface_hub import hf_hub_download, notebook_login

In [4]:
sae_name = "DeepSeek-R1-Distill-Llama-8B-SAE-l19"
# sae_name = "Llama-3.2-1B-Instruct-SAE-l9"
# sae_name = "DeepSeek-R1-Distill-Llama-70B-SAE-l48"

In [5]:
if sae_name == "Llama-3.2-1B-Instruct-SAE-l9":
    notebook_login()

In [6]:
file_path = hf_hub_download(
    repo_id=f"qresearch/{sae_name}",
    filename=f"{sae_name}.pt",
    repo_type="model"
)

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = ("deepseek-ai/DeepSeek-R1-Distill-Llama-8B" if sae_name == "DeepSeek-R1-Distill-Llama-8B-SAE-l19"
              else "meta-llama/Llama-3.2-1B-Instruct" if sae_name == "Llama-3.2-1B-Instruct-SAE-l9"
              else "deepseek-ai/DeepSeek-R1-Distill-Llama-70B")
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="bfloat16", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

expansion_factor = 8 if sae_name == "DeepSeek-R1-Distill-Llama-70B-SAE-l48" else 16
sae = SparseAutoencoder.from_pretrained(
    path=file_path,
    input_dim=model.config.hidden_size,
    expansion_factor=expansion_factor,
    device="mps"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/var/folders/bt/h53kb9_n1yb13w1c6j1f04v80000gp/T/ipykernel_17387/4177196090.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=d

In [29]:
inputs = tokenizer.apply_chat_template(
    [
        {"role": "user", "content": "Can penguins fly? Segment the thinking process into clear steps. Indicate \"YES\" or \"NO\" at the end of the steps."},
    ],
    add_generation_prompt=True,
    return_tensors="pt",
).to("mps")
outputs = model.generate(input_ids=inputs, max_new_tokens=1200)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<｜begin▁of▁sentence｜><｜User｜>Can penguins fly? Segment the thinking process into clear steps. Indicate "YES" or "NO" at the end of the steps.<｜Assistant｜><think>
Okay, so the question is, can penguins fly? Hmm, I need to figure this out step by step. Let me start by thinking about what I know about penguins. I remember that penguins are birds, right? So, birds can fly, or at least some of them can. But wait, not all birds can fly. For example, ostriches and emus can't fly, but they have wings. So, maybe penguins can fly too, but I'm not sure.

I should consider the physical characteristics of penguins. They live in the cold, mostly in Antarctica. Their bodies are adapted for swimming, with their wings being flat and webbed. Oh, wait, penguins don't fly in the traditional sense like birds do. They don't have feathers on their wings like other birds. Instead, their wings are more like flippers. So, if their wings are flippers, maybe they don't use them to fly. That makes me think penguin

In [30]:
outputs.shape

torch.Size([1, 657])

In [9]:
def gather_residual_activations(model, target_layer, inputs):
    target_act = None
    def gather_target_act_hook(mod, inputs, outputs):
        nonlocal target_act
        target_act = inputs[0]  # Get residual stream from layer input
        return outputs

    handle = model.model.layers[target_layer].register_forward_hook(gather_target_act_hook)
    with torch.no_grad():
        _ = model(inputs)
    handle.remove()
    return target_act

In [10]:
layer_id = (19 if sae_name == "DeepSeek-R1-Distill-Llama-8B-SAE-l19"
            else 9 if sae_name == "Llama-3.2-1B-Instruct-SAE-l9"
            else 48)

In [11]:
target_act = gather_residual_activations(model, layer_id, inputs)

In [12]:
def ensure_same_device(sae, target_act):
    """Ensure SAE and activations are on the same device"""
    model_device = target_act.device
    sae = sae.to(model_device)
    return sae, target_act.to(model_device)

sae, target_act = ensure_same_device(sae, target_act)
sae_acts = sae.encode(target_act.to(torch.float32))
recon = sae.decode(sae_acts)

In [13]:
var_explained = 1 - torch.mean((recon - target_act.to(torch.float32)) ** 2) / torch.var(target_act.to(torch.float32))
print(f"Variance explained: {var_explained:.3f}")

Variance explained: 0.993


In [27]:
inputs = tokenizer.apply_chat_template(
    [
        {"role": "user", "content": "Roleplay as a pirate"},
        {"role": "assistant", "content": "Yarr, I'll be speakin' like a true seafarer from here on out! Got me sea legs ready and me vocabulary set to proper pirate speak. What can I help ye with, me hearty?"},
    ],
    return_tensors="pt",
).to("mps")

In [28]:
import numpy as np
# Get activations
target_act = gather_residual_activations(model, layer_id, inputs)
sae_acts = sae.encode(target_act.to(torch.float32))

# Get token IDs and decode them for reference
tokens = inputs[0].cpu().numpy()
token_texts = tokenizer.convert_ids_to_tokens(tokens)

# Find which tokens are part of the assistant's response
token_ids = inputs[0].cpu().numpy()
is_special = (token_ids >= 128000) & (token_ids <= 128255)
special_positions = np.where(is_special)[0]

assistant_start = special_positions[-2] + 1
assistant_tokens = slice(assistant_start, None)

# Get activation statistics for assistant's response
assistant_activations = sae_acts[0, assistant_tokens]
mean_activations = assistant_activations.mean(dim=0)

# Find top activated features during pirate speech
num_top_features = 20
top_features = mean_activations.topk(num_top_features)

print("Top activated features during pirate speech:")
for idx, value in zip(top_features.indices, top_features.values):
    print(f"Feature {idx}: {value:.3f}")

# Look at how these features activate across different tokens
print("\nActivation patterns across tokens:")
for i, (token, acts) in enumerate(zip(token_texts[assistant_tokens], assistant_activations)):
    top_acts = acts[top_features.indices]
    if top_acts.max() > 0.2:  # Only show tokens with significant activation
        print(f"\nToken: {token}")
        for feat_idx, act_val in zip(top_features.indices, top_acts):
            if act_val > 0.2:  # Threshold for "active" features
                print(f"  Feature {feat_idx}: {act_val:.3f}")


Top activated features during pirate speech:
Feature 7560: 1.677
Feature 34357: 0.742
Feature 14322: 0.559
Feature 19958: 0.311
Feature 37063: 0.299
Feature 53431: 0.247
Feature 34347: 0.234
Feature 1262: 0.221
Feature 43713: 0.217
Feature 55236: 0.190
Feature 54314: 0.190
Feature 60475: 0.190
Feature 53323: 0.186
Feature 36282: 0.177
Feature 18706: 0.170
Feature 60476: 0.149
Feature 61938: 0.149
Feature 56551: 0.142
Feature 266: 0.141
Feature 29635: 0.137

Activation patterns across tokens:

Token: Y
  Feature 7560: 2.326
  Feature 34357: 0.570
  Feature 34347: 0.578
  Feature 1262: 0.288
  Feature 53323: 0.347

Token: arr
  Feature 7560: 2.790
  Feature 34357: 0.685
  Feature 60475: 0.461
  Feature 53323: 0.472
  Feature 266: 0.268

Token: ,
  Feature 7560: 3.592
  Feature 34357: 2.499
  Feature 19958: 0.335
  Feature 1262: 0.468
  Feature 60475: 0.212
  Feature 53323: 0.604
  Feature 60476: 0.571

Token: ĠI
  Feature 7560: 1.948
  Feature 34357: 1.654
  Feature 14322: 0.321
  Featur

In [25]:
def generate_with_intervention(
    model,
    tokenizer,
    sae,
    messages: list[dict],
    feature_idx: int,
    intervention: float = 3.0,
    target_layer: int = 9,
    max_new_tokens: int = 50
):
    modified_activations = None

    def intervention_hook(module, inputs, outputs):
        nonlocal modified_activations
        activations = inputs[0]

        features = sae.encode(activations.to(torch.float32))
        reconstructed = sae.decode(features)
        error = activations.to(torch.float32) - reconstructed

        features[:, :, feature_idx] += intervention

        modified = sae.decode(features) + error
        modified_activations = modified
        modified_activations = modified.to(torch.bfloat16)

        return outputs

    def output_hook(module, inputs, outputs):
        nonlocal modified_activations
        if modified_activations is not None:
            return (modified_activations,) + outputs[1:] if len(outputs) > 1 else (modified_activations,)
        return outputs

    handles = [
        model.model.layers[target_layer].register_forward_hook(intervention_hook),
        model.model.layers[target_layer].register_forward_hook(output_hook)
    ]

    try:
        input_tokens = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

        outputs = model.generate(
            input_tokens,
            max_new_tokens=max_new_tokens,
            do_sample=False  # Use greedy decoding for consistency
        )

        generated_text = tokenizer.decode(outputs[0])

    finally:
        for handle in handles:
            handle.remove()

    return generated_text

messages = [
    {"role": "user", "content": "How are you doing?"}
]
feature_to_modify = 7560

print("Original generation:")
input_tokens = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)
outputs = model.generate(input_tokens, max_new_tokens=1000, do_sample=False)
print(tokenizer.decode(outputs[0]))

print("\nGeneration with modified feature:")
modified_text = generate_with_intervention(
    model=model,
    tokenizer=tokenizer,
    sae=sae,
    messages=messages,
    feature_idx=feature_to_modify,
    intervention=10,
    target_layer=layer_id,
    max_new_tokens=100
)
print(modified_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Original generation:


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<｜begin▁of▁sentence｜><｜User｜>How are you doing?<｜Assistant｜><think>
Okay, so I'm trying to figure out how to respond to the question "How are you doing?" I know that this is a common question people ask when they want to start a conversation or show they care. But I'm a bit confused about the best way to answer it, especially since I'm an AI and not a human. Let me think through this step by step.

First, I should consider the context in which the question is being asked. If someone is asking me how I'm doing, they might be looking for a friendly exchange. Since I don't have feelings or emotions, I shouldn't pretend to have them. Instead, I should acknowledge that I'm an AI and explain how I function.

I remember that in previous interactions, I've responded by saying I'm just a program without feelings, but I want to make sure that's the right approach. Maybe I can also mention that I'm here to help them with whatever they need. That way, the conversation can shift towards assisting t

In [26]:

print("\nGeneration with modified feature:")

messages = [
    {"role": "user", "content": "How many Rs in strawberry?"}
]

modified_text = generate_with_intervention(
    model=model,
    tokenizer=tokenizer,
    sae=sae,
    messages=messages,
    feature_idx=feature_to_modify,
    intervention=8,
    target_layer=layer_id,
    max_new_tokens=1000
)
print(modified_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Generation with modified feature:
<｜begin▁of▁sentence｜><｜User｜>How many Rs in strawberry?<｜Assistant｜><think>
The question "How many Rs in strawberry?" is a bit tricky because it's not a straightforward word to count letters on.

First, I'll break down the word "strawberry" letter by letter.

S, T, R, A, W, B, E, R, R, I, G.

Looking at each letter, I see there are three 'R's in the word.

So, the number of 'R's in "strawberry" is three.
</think>

To determine how many **Rs** are in the word "strawberry," let's break it down step by step.

**Step 1:** Write down the word:

```
S T R A W B E R R I G
```

**Step 2:** Identify each letter and count the number of **Rs**.

- **S** - 1
- **T** - 1
- **R** - 1 (First R)
- **A** - 1
- **W** - 1
- **B** - 1
- **E** - 1
- **R** - 2 (Second R)
- **R** - 3 (Third R)
- **I** - 1
- **G** - 1

**Step 3:** Count the total number of **Rs**:

There are **3** Rs in the word "strawberry."

\[
\boxed{3}
\]<｜end▁of▁sentence｜>
